# Week 8: Mission & Behavior Planning

### Topics Covered

- Route Planning (Dijkstra, A* search on road network graphs); Finite State Machines (FSM) for Decision Making (e.g., lane change, stop, wait)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque, defaultdict
import heapq
from enum import Enum
from typing import List, Tuple, Dict, Set

# Set random seed for reproducibility
np.random.seed(42)

## 1. Route Planning on Road Networks

Route planning determines the **high-level path** from origin to destination on a road network. This is the "mission" level - deciding which roads to take.

### Road Network as a Graph

**Nodes (Vertices):** Intersections, decision points
**Edges:** Road segments connecting nodes
**Weights:** Travel time, distance, or cost

### Graph Representation

$$G = (V, E)$$

Where:
- $V$: Set of vertices (nodes)
- $E$: Set of edges connecting vertices
- $w(u, v)$: Weight/cost of edge from $u$ to $v$

### Problem Formulation

**Input:**
- Graph $G = (V, E)$ with edge weights
- Start node $s$
- Goal node $g$

**Output:**
- Path $P = [s, v_1, v_2, \ldots, g]$ that minimizes total cost

**Cost Function:**
$$\text{cost}(P) = \sum_{i=0}^{n-1} w(v_i, v_{i+1})$$

### Applications in Autonomous Driving

1. **Navigation**: Route from current location to destination
2. **Rerouting**: Dynamic replanning when roads are blocked
3. **Multi-objective**: Balance time, distance, fuel, safety
4. **Time-dependent**: Account for traffic predictions

In [ ]:
# Finite State Machine for Lane Change Behavior

class DrivingState(Enum):
    """Enum for driving states"""
    LANE_KEEPING = "Lane Keeping"
    PREP_LANE_CHANGE_LEFT = "Prepare Lane Change Left"
    LANE_CHANGE_LEFT = "Executing Lane Change Left"
    PREP_LANE_CHANGE_RIGHT = "Prepare Lane Change Right"
    LANE_CHANGE_RIGHT = "Executing Lane Change Right"
    EMERGENCY_STOP = "Emergency Stop"


class HighwayBehaviorFSM:
    """Finite State Machine for highway driving behavior"""
    
    def __init__(self):
        self.state = DrivingState.LANE_KEEPING
        self.lane = 1  # Current lane (0=left, 1=center, 2=right on 3-lane highway)
        self.state_history = [self.state]
        self.lane_change_timer = 0
        self.lane_change_duration = 30  # steps to complete lane change
        
    def update(self, ego_speed, front_vehicle_speed, front_vehicle_dist,
               left_lane_clear, right_lane_clear, emergency):
        """
        Update FSM based on current conditions
        
        Parameters:
        - ego_speed: Speed of our vehicle
        - front_vehicle_speed: Speed of vehicle ahead
        - front_vehicle_dist: Distance to vehicle ahead
        - left_lane_clear: Is left lane safe for lane change?
        - right_lane_clear: Is right lane safe for lane change?
        - emergency: Emergency condition detected?
        """
        old_state = self.state
        
        # Emergency override
        if emergency:
            self.state = DrivingState.EMERGENCY_STOP
        
        # State machine logic
        elif self.state == DrivingState.LANE_KEEPING:
            # Check if vehicle ahead is slow
            if front_vehicle_dist < 50 and front_vehicle_speed < ego_speed - 5:
                # Try to change lanes
                if self.lane > 0 and left_lane_clear:
                    self.state = DrivingState.PREP_LANE_CHANGE_LEFT
                elif self.lane < 2 and right_lane_clear:
                    self.state = DrivingState.PREP_LANE_CHANGE_RIGHT
        
        elif self.state == DrivingState.PREP_LANE_CHANGE_LEFT:
            if left_lane_clear:
                # Safe to change
                self.state = DrivingState.LANE_CHANGE_LEFT
                self.lane_change_timer = 0
            else:
                # Not safe anymore, abort
                self.state = DrivingState.LANE_KEEPING
        
        elif self.state == DrivingState.LANE_CHANGE_LEFT:
            self.lane_change_timer += 1
            if self.lane_change_timer >= self.lane_change_duration:
                # Lane change complete
                self.lane -= 1
                self.state = DrivingState.LANE_KEEPING
                self.lane_change_timer = 0
        
        elif self.state == DrivingState.PREP_LANE_CHANGE_RIGHT:
            if right_lane_clear:
                self.state = DrivingState.LANE_CHANGE_RIGHT
                self.lane_change_timer = 0
            else:
                self.state = DrivingState.LANE_KEEPING
        
        elif self.state == DrivingState.LANE_CHANGE_RIGHT:
            self.lane_change_timer += 1
            if self.lane_change_timer >= self.lane_change_duration:
                self.lane += 1
                self.state = DrivingState.LANE_KEEPING
                self.lane_change_timer = 0
        
        elif self.state == DrivingState.EMERGENCY_STOP:
            # Stay in emergency stop until resolved
            if not emergency:
                self.state = DrivingState.LANE_KEEPING
        
        # Record state transition
        if old_state != self.state:
            self.state_history.append(self.state)
            print(f"State transition: {old_state.value} → {self.state.value}")
        
        return self.get_action()
    
    def get_action(self):
        """Return action based on current state"""
        if self.state == DrivingState.LANE_KEEPING:
            return "maintain_speed"
        elif self.state == DrivingState.PREP_LANE_CHANGE_LEFT:
            return "signal_left"
        elif self.state == DrivingState.LANE_CHANGE_LEFT:
            return "steer_left"
        elif self.state == DrivingState.PREP_LANE_CHANGE_RIGHT:
            return "signal_right"
        elif self.state == DrivingState.LANE_CHANGE_RIGHT:
            return "steer_right"
        elif self.state == DrivingState.EMERGENCY_STOP:
            return "brake_hard"
        return "maintain_speed"


# Simulate highway driving scenario
def simulate_highway_driving():
    fsm = HighwayBehaviorFSM()
    
    # Simulation parameters
    num_steps = 200
    ego_speed = 30  # m/s
    
    # Data recording
    states_over_time = []
    lanes_over_time = []
    actions_over_time = []
    
    print("Starting highway driving simulation...\n")
    
    for t in range(num_steps):
        # Simulate environment (simple scenario)
        if t < 50:
            # No obstacles
            front_vehicle_speed = ego_speed
            front_vehicle_dist = 100
            left_lane_clear = True
            right_lane_clear = True
            emergency = False
        elif t < 100:
            # Slow vehicle ahead
            front_vehicle_speed = 20
            front_vehicle_dist = 40
            left_lane_clear = True
            right_lane_clear = False
            emergency = False
        elif t < 150:
            # Back in left lane, slow vehicle in left lane
            front_vehicle_speed = 25
            front_vehicle_dist = 45
            left_lane_clear = False
            right_lane_clear = True
            emergency = False
        else:
            # Clear road
            front_vehicle_speed = ego_speed
            front_vehicle_dist = 100
            left_lane_clear = True
            right_lane_clear = True
            emergency = False
        
        # Update FSM
        action = fsm.update(ego_speed, front_vehicle_speed, front_vehicle_dist,
                           left_lane_clear, right_lane_clear, emergency)
        
        # Record
        states_over_time.append(fsm.state)
        lanes_over_time.append(fsm.lane)
        actions_over_time.append(action)
    
    # Visualization
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # State over time
    ax1 = axes[0]
    state_values = [list(DrivingState).index(s) for s in states_over_time]
    state_names = [s.value for s in DrivingState]
    
    ax1.plot(state_values, 'b-', linewidth=2)
    ax1.set_yticks(range(len(DrivingState)))
    ax1.set_yticklabels(state_names, fontsize=9)
    ax1.set_ylabel('Driving State')
    ax1.set_title('FSM State Over Time')
    ax1.grid(True, alpha=0.3)
    
    # Lane over time
    ax2 = axes[1]
    ax2.plot(lanes_over_time, 'g-', linewidth=2)
    ax2.set_yticks([0, 1, 2])
    ax2.set_yticklabels(['Left Lane', 'Center Lane', 'Right Lane'])
    ax2.set_ylabel('Current Lane')
    ax2.set_xlabel('Time Step')
    ax2.set_title('Lane Position Over Time')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(-0.5, 2.5)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nSimulation complete!")
    print(f"State transitions: {len(fsm.state_history) - 1}")
    print(f"Final lane: {['Left', 'Center', 'Right'][fsm.lane]}")

simulate_highway_driving()

## Exercises

### Exercise 1: Bidirectional A* Search

Implement bidirectional A* which searches from both start and goal simultaneously, meeting in the middle.

**Tasks:**
1. Implement bidirectional A* search
2. Compare with standard A* in terms of nodes explored
3. Test on larger road networks
4. Analyze when bidirectional search helps most

**Hint:** Run two A* searches simultaneously and stop when they meet.

In [ ]:
# Exercise 3 - Template

def time_dependent_weight(edge, current_time):
    """
    TODO: Return edge weight based on time of day
    
    Example: Highway edges are slower during rush hour
    """
    pass

def time_dependent_astar(network, start, goal, departure_time):
    """
    TODO: Implement A* with time-dependent weights
    
    Track current time as search progresses
    """
    pass

# Your implementation here

## References and Additional Resources

### Core Textbooks

1. **Artificial Intelligence: A Modern Approach** by Russell & Norvig (4th Edition, 2020)
   - Chapter 3: Solving Problems by Searching
   - Dijkstra's, A*, and heuristic search

2. **Planning Algorithms** by LaValle (2006)
   - Comprehensive coverage of motion planning
   - Available free online: http://planning.cs.uiuc.edu/

3. **Automated Planning and Acting** by Ghallab, Nau, & Traverso (2016)
   - FSM, hierarchical task networks, decision-theoretic planning

### Key Papers

1. **A* Search**
   - Hart, Nilsson, & Raphael (1968) - "A Formal Basis for the Heuristic Determination of Minimum Cost Paths"
   - Original A* paper

2. **Behavior Planning**
   - Ulbrich & Maurer (2013) - "Probabilistic Online POMDP Decision Making for Lane Changes"
   - Wei et al. (2013) - "Towards a Viable Autonomous Driving Research Platform"

3. **Hierarchical Planning**
   - Kaelbling & Lozano-Pérez (2013) - "Integrated Task and Motion Planning in Belief Space"

### Algorithms & Data Structures

1. **Graph Algorithms**
   - Priority queue (heap) for Dijkstra/A*
   - Bidirectional search
   - D* Lite for dynamic replanning

2. **Behavior Planning Approaches**
   - Finite State Machines (FSM)
   - Behavior trees
   - Markov Decision Processes (MDP)
   - Partially Observable MDPs (POMDP)

### Software & Tools

1. **Route Planning Libraries**
   - **NetworkX** (Python): Graph algorithms - https://networkx.org/
   - **OSRM**: Open Source Routing Machine - http://project-osrm.org/
   - **Graphhopper**: Java routing engine - https://www.graphhopper.com/

2. **Behavior Planning**
   - **py_trees**: Behavior trees in Python - https://github.com/splintered-reality/py_trees
   - **SMACH**: ROS state machine library

3. **Simulation**
   - **CARLA**: Open-source autonomous driving simulator
   - **SUMO**: Traffic simulation

### Online Resources

1. **Coursera: Motion Planning for Self-Driving Cars**
   - University of Toronto
   - Covers A*, lattice planning, behavior planning

2. **MIT OpenCourseWare: Artificial Intelligence**
   - Search algorithms and planning

3. **Red Blob Games: A* Tutorial**
   - https://www.redblobgames.com/pathfinding/a-star/
   - Interactive visualizations

### Applications in Autonomous Vehicles

1. **Mission Planning Hierarchy**
   - **Mission**: Route from A to B (Dijkstra/A*)
   - **Behavior**: Lane changes, merging, intersection handling (FSM)
   - **Motion**: Trajectory generation (next week's topic)

2. **Real-World Considerations**
   - Dynamic replanning (road closures, accidents)
   - Multi-objective optimization (time, safety, comfort, fuel)
   - Uncertainty handling
   - Real-time constraints

3. **Industry Practices**
   - Waymo: Hierarchical planning with prediction
   - Tesla: Neural network-based planning
   - Cruise: Rule-based + learning hybrid

### Advanced Topics

1. **Learning-Based Planning**
   - Imitation learning for behavior cloning
   - Reinforcement learning for policy optimization
   - Inverse reinforcement learning for cost function learning

2. **Probabilistic Planning**
   - Monte Carlo Tree Search (MCTS)
   - POMDP solvers
   - Contingency planning

3. **Multi-Agent Planning**
   - Game theory for interaction
   - Joint planning with other vehicles
   - Negotiation and communication

---

## Summary

This notebook covered mission and behavior planning for autonomous vehicles:

**Key Concepts:**
- Route planning on road networks (graphs)
- Dijkstra's algorithm for shortest paths
- A* search with heuristics for efficient planning
- Finite State Machines for behavior decision-making

**Practical Skills:**
- Implementing graph-based search algorithms
- Designing FSMs for driving behaviors
- Comparing algorithm efficiency
- Handling state transitions

**Next Steps:**
1. Study trajectory generation and motion planning
2. Explore learning-based behavior planning
3. Implement behavior trees for complex scenarios
4. Learn about POMDP for uncertainty handling